In [1]:
from tfedlrn.proto.message_pb2 import *

/home/msheller/.local/share/virtualenvs/tfedlrn-CPfKUQY7/lib/python3.5/site-packages/google/protobuf/__init__.py:37: UserWarning: Module tfedlrn was already imported from /home/msheller/git/tfedlrn/src/tfedlrn/__init__.py, but /home/msheller/.local/share/virtualenvs/tfedlrn-CPfKUQY7/lib/python3.5/site-packages/tfedlrn-0.0.0-py3.5.egg is being added to sys.path
  __import__('pkg_resources').declare_namespace(__name__)


In [2]:
def serialize(message):
    flm = FLMessage(**{message.__class__.__name__.lower():message})
    return flm.SerializeToString()


def deserialize(s):
    flmessage = FLMessage.FromString(s)
    return getattr(flmessage, flmessage.WhichOneof('payload'))

In [3]:
jobreq = JobRequest(header=MessageHeader(sender='hi', recipient='you', federation_id='fedit!', counter=0),
                    model_header=ModelHeader(id='model!!!', version=-1))

In [4]:
jobreq

header {
  sender: "hi"
  recipient: "you"
  federation_id: "fedit!"
  counter: 0
}
model_header {
  id: "model!!!"
  version: -1
}

In [5]:
deserialize(serialize(jobreq))

header {
  sender: "hi"
  recipient: "you"
  federation_id: "fedit!"
  counter: 0
}
model_header {
  id: "model!!!"
  version: -1
}

In [7]:
import abc
import zmq

class ZMQConnection(metaclass=abc.ABCMeta):

    def __init__(self,
                 name,
                 server_addr='127.0.0.1',
                 server_port=5555,
                 compression_func=None,
                 decompression_func=None,
                 receive_timeout=-1):
        self.name = name
        self.server_addr = 'tcp://{}:{}'.format(server_addr, server_port)
        self.context = zmq.Context()
        self.context.setsockopt(zmq.RCVTIMEO, receive_timeout)
        self.socket = None
        self.compression_func = compression_func
        self.decompression_func = decompression_func

    @abc.abstractmethod
    def _connect(self):
        pass

    def send(self, message):
        if self.socket is None:
            self._connect()

        print('{} sending {}'.format(self, message.__class__.__name__))
        # if self.compression_func is not None:
        #     message = self.compression_func(message)
        message = serialize(message)
        self.socket.send(message)

    def receive(self):
        if self.socket is None:
            self._connect()

        message = self.socket.recv()
        # if self.decompression_func is not None:
        #     message = self.decompression_func(message)
        message = deserialize(message)
        print('{} received {}'.format(self, message.__class__.__name__))
        return message

    def __repr__(self):
        return '{}: {}'.format(self.__class__.__name__, self.name)

    def __str__(self):
        return self.__repr__()


class ZMQServer(ZMQConnection):

    def _connect(self):
        print('{} binding to socket: {}'.format(self, self.server_addr))
        context = zmq.Context()
        self.socket = context.socket(zmq.REP)
        self.socket.bind(self.server_addr)


class ZMQClient(ZMQConnection):

    def _connect(self):
        print('{} connecting to server: {}'.format(self, self.server_addr))
        context = zmq.Context()
        self.socket = context.socket(zmq.REQ)
        self.socket.connect(self.server_addr)


In [8]:
server = ZMQServer('server')

In [9]:
client = ZMQClient('client')

In [10]:
client.send(jobreq)

ZMQClient: client connecting to server: tcp://127.0.0.1:5555
ZMQClient: client sending JobRequest


In [11]:
server.receive()

ZMQServer: server binding to socket: tcp://127.0.0.1:5555
ZMQServer: server received JobRequest


header {
  sender: "hi"
  recipient: "you"
  federation_id: "fedit!"
  counter: 0
}
model_header {
  id: "model!!!"
  version: -1
}